In [137]:
import pandas as pd
import altair as alt

AreaPlanted = pd.read_csv("NFD - Area planted by ownership and species group - EN FR.csv", encoding='latin-1')
AreaSeeded = pd.read_csv("NFD - Area of direct seeding by ownership and application method - EN FR.csv", encoding='latin-1')
AreaHarvested = pd.read_csv("NFD - Area harvested by ownership and harvesting method - EN FR.csv", encoding='latin-1')
AreaInsectDamage = pd.read_csv("NFD - Area of moderate to severe defoliation (including beetle-killed trees) by insects - EN FR.csv", encoding='latin-1')

# Data Source Sitation  
# Canadian Council of Forest Ministers - Conseil canadien des ministres des forêts. (2020). National Forestry Database -
# Base de données nationales des forêts - Canada (Version 2.0.0) [Data set]. Natural Resources Canada – Ressources naturelles Canada. 
# http://doi.org/10.5281/zenodo.3690046

#Remove icky French and other un-needed columns
AreaPlanted = AreaPlanted.drop(["Année", "Juridiction","Groupe d'espèces","Tenure (Fr)","Superficie (en hectare)",
                                "Qualificatifs de données","Data qualifier","Species group","ISO","Tenure (En)"],
                               axis=1).rename(columns={'Area (hectares)': 'Area Planted (Hectares)','Jurisdiction': 'Province/Territory'})
AreaPlanted = AreaPlanted.groupby(by=["Year","Province/Territory"],as_index=False).sum()
print("\n\nArea Planted")
print("---------------------------------------------------------------------------\n")
print(AreaPlanted)

AreaSeeded = AreaSeeded.drop(["Année", "Juridiction","Méthode d'application","Tenure (Fr)","Superficie (en hectare)",
                              "Qualificatifs de données","Data qualifier","ISO","Tenure (En)","Application Method"], 
                             axis=1).rename(columns={'Area (hectares)': 'Area Seeded (Hectares)','Jurisdiction': 'Province/Territory'})
AreaSeeded = AreaSeeded.groupby(by=["Year","Province/Territory"],as_index=False).sum()
print("\n\nArea Seeded")
print("---------------------------------------------------------------------------\n")
print(AreaSeeded)

AreaHarvested = AreaHarvested.drop(["Année", "Juridiction","Aménagement","Méthode de récolte","Tenure (Fr)",
                                    "Superficie (en hectare)","Qualificatifs de données","Data qualifier","ISO","Management",
                                    "Harvesting method","Tenure (En)"], axis=1).rename(columns={'Area (hectares)': 'Area Harvested (Hectares)',
                                                                                                'Jurisdiction': 'Province/Territory'})
AreaHarvested = AreaHarvested.groupby(by=["Year","Province/Territory"],as_index=False).sum()
print("\n\nArea Harvested")
print("---------------------------------------------------------------------------\n")
print(AreaHarvested)

AreaInsectDamage = AreaInsectDamage.drop(["Année", "Juridiction","Insecte espèce","Superficie défoliée (en hectare)",
                                          "Qualificatifs de données","Data qualifier","ISO","Insect species"],
                                         axis=1).rename(columns={'Defoliated area (hectares)': 'Area Insects Damaged (hectares)',
                                                                 'Jurisdiction': 'Province/Territory'})
AreaInsectDamage = AreaInsectDamage.groupby(by=["Year","Province/Territory"],as_index=False).sum()
print("\n\nArea Insect Damage")
print("---------------------------------------------------------------------------\n")
print(AreaInsectDamage)






Area Planted
---------------------------------------------------------------------------

     Year         Province/Territory  Area Planted (Hectares)
0    1975                    Alberta                   5487.0
1    1975           British Columbia                  63078.0
2    1975                   Manitoba                   1510.0
3    1975              New Brunswick                   6675.0
4    1975  Newfoundland and Labrador                      0.0
..    ...                        ...                      ...
576  2020                    Ontario                  43219.0
577  2020       Prince Edward Island                    369.0
578  2020                     Quebec                  64868.0
579  2020               Saskatchewan                   5522.0
580  2020                      Yukon                     14.0

[581 rows x 3 columns]


Area Seeded
---------------------------------------------------------------------------

     Year    Province/Territory  Area Seeded (Hec

In [138]:
data = pd.merge(AreaPlanted, AreaSeeded,on=['Year','Province/Territory'],how='outer').merge(
    AreaHarvested, on=['Year','Province/Territory'],how='outer').merge(
    AreaInsectDamage, on=['Year','Province/Territory'],how='outer')

data.fillna(0, inplace=True)

data = data.assign(Area_Reforested = data['Area Planted (Hectares)'] + data['Area Seeded (Hectares)'],
                  Area_Deforested = data['Area Harvested (Hectares)'] + data['Area Insects Damaged (hectares)']).rename(
    columns={'Area_Reforested': 'Area Reforested', 'Area_Deforested':'Area Deforested'})
data = data.assign(Net_Area = data['Area Reforested'] - data['Area Deforested'],
                  Net_Area_Loss = data['Area Harvested (Hectares)']-data['Area Reforested']).rename(columns={'Net_Area':'Net Area','Net_Area_Loss':'Deforestation'})    

neg_vs_pos = []
for i in range(len(data)):
    if data['Net Area'][i] > 0:
        neg_vs_pos.append(1)
    else:
        neg_vs_pos.append(0)
print(sum(neg_vs_pos))


harv_vs_pos = []
for i in range(len(data)):
    if (data['Area Reforested'][i] - data['Area Harvested (Hectares)'][i])> 0:
        harv_vs_pos.append(1)
    else:
        harv_vs_pos.append(0)
print('Out of '+str(len(data))+' entries, ' + str(sum(harv_vs_pos)) + " are positive reforestation by province years. This exludes fire and insect loss.\nThat ratio is "
      + str( round( (sum(harv_vs_pos) /len(data) ),3 ) ) )

#data['Area Harvested (Hectares)'] = data['Area Harvested (Hectares)'].transform(lambda x: x/100000)
#data['Area Reforested'] = data['Area Reforested'].transform(lambda x: x/100000)
#data['Area Deforested'] = data['Area Deforested'].transform(lambda x: x/100000)
#data['Net Area'] = data['Net Area'].transform(lambda x: x/100000)
#data['Area Insects Damaged (hectares)'] = data['Area Insects Damaged (hectares)'].transform(lambda x: x/100000)
#data['Harvest vs. Reforestation'] = data['Harvest vs. Reforestation'].transform(lambda x: x/100000)
#data['Area Planted (Hectares)'] = data['Area Planted (Hectares)'].transform(lambda x: x/100000)
#data['Area Seeded (Hectares)'] = data['Area Seeded (Hectares)'].transform(lambda x: x/100000)
data['Year'] = pd.to_datetime(data['Year'],format='%Y')

data

14
Out of 581 entries, 138 are positive reforestation by province years. This exludes fire and insect loss.
That ratio is 0.238


,Year,Province/Territory,Area Planted (Hectares),Area Seeded (Hectares),Area Harvested (Hectares),Area Insects Damaged (hectares),Area Reforested,Area Deforested,Net Area,Deforestation
0,1975-01-01,Alberta,5487.0,10235.0,21682.0,6000.0,15722.0,27682.0,-11960.0,5960.0
1,1975-01-01,British Columbia,63078.0,0.0,156976.0,30000.0,63078.0,186976.0,-123898.0,93898.0
2,1975-01-01,Manitoba,1510.0,223.0,0.0,20000.0,1733.0,20000.0,-18267.0,-1733.0
3,1975-01-01,New Brunswick,6675.0,125.0,0.0,3500000.0,6800.0,3500000.0,-3493200.0,-6800.0
4,1975-01-01,Newfoundland and Labrador,0.0,0.0,0.0,1903000.0,0.0,1903000.0,-1903000.0,0.0
...,...,...,...,...,...,...,...,...,...,...
576,2020-01-01,Ontario,43219.0,4510.0,125456.0,2114198.0,47729.0,2239654.0,-2191925.0,77727.0
577,2020-01-01,Prince Edward Island,369.0,0.0,2652.0,0.0,369.0,2652.0,-2283.0,2283.0
578,2020-01-01,Quebec,64868.0,0.0,198990.0,6246502.0,64868.0,6445492.0,-6380624.0,134122.0
579,2020-01-01,Saskatchewan,5522.0,0.0,22968.0,38507.0,5522.0,61475.0,-55953.0,17446.0


In [139]:

data = data.rename(columns={'Area Harvested (Hectares)':'Area Harvested'})

data1 = data.melt(id_vars=['Year','Province/Territory'],
                  value_vars=['Area Reforested','Area Harvested'])

data1 = data1.rename(columns={'variable' :'Silvicultural Activity'})

#hectares to acres
data1['value'] = data1['value'].transform(lambda x:x*2.471054/100000)
data1 = data1.rename(columns={'Province/Territory':'Provinces'})

data1['Provinces'] = pd.Categorical(data1['Provinces'], ['British Columbia','Alberta','Saskatchewan','Manitoba','Ontario','Quebec','New Brunswick',
       'Prince Edward Island','Nova Scotia','Newfoundland and Labrador'])
data1=data1.sort_values(by = ['Year', 'Provinces'], 
                           ascending=[True, True], 
                           inplace=False)
#remove terriotires as data isn't significant
data1 = data1.dropna()
data1 = data1.round(2)
data1.head(10
          )

,Year,Provinces,Silvicultural Activity,value
1,1975-01-01,British Columbia,Area Reforested,1.56
582,1975-01-01,British Columbia,Area Harvested,3.88
0,1975-01-01,Alberta,Area Reforested,0.39
581,1975-01-01,Alberta,Area Harvested,0.54
10,1975-01-01,Saskatchewan,Area Reforested,0.09
591,1975-01-01,Saskatchewan,Area Harvested,0.00
2,1975-01-01,Manitoba,Area Reforested,0.04
583,1975-01-01,Manitoba,Area Harvested,0.00
7,1975-01-01,Ontario,Area Reforested,1.32
588,1975-01-01,Ontario,Area Harvested,0.00


In [140]:
data2=data.melt(id_vars=['Year','Province/Territory'],
                  value_vars=['Deforestation'])
data2['value'] = data2['value'].transform(lambda x:x*2.471054)
data2 = data2.rename(columns={'variable' :'Silvicultural Activity','Province/Territory':'Provinces','Reforestation Deficit':'Deforestation'})

data2['Provinces'] = pd.Categorical(data2['Provinces'], ['British Columbia','Alberta','Saskatchewan','Manitoba','Ontario','Quebec','New Brunswick',
       'Prince Edward Island','Nova Scotia','Newfoundland and Labrador'])
data2=data2.sort_values(by = ['Year', 'Provinces'], 
                           ascending=[True, True], 
                           inplace=False)
#remove terriotires as data isn't significant
data2 = data2.dropna()
data2 = data2.round(0)
print(data2)
simp_data2 = data2.groupby(by=['Year','Silvicultural Activity'],as_index=False)['value'].sum()
data2.head(12)


simp_data2['value']= simp_data2['value'].cumsum()
simp_data2['value']= simp_data2['value'].transform(lambda x:x/1000000)
simp_data2['value']= simp_data2['value'].round(2)
simp_data2.head(5)

          Year                  Provinces Silvicultural Activity     value
1   1975-01-01           British Columbia          Deforestation  232027.0
0   1975-01-01                    Alberta          Deforestation   14727.0
10  1975-01-01               Saskatchewan          Deforestation   -9051.0
2   1975-01-01                   Manitoba          Deforestation   -4282.0
7   1975-01-01                    Ontario          Deforestation -132350.0
..         ...                        ...                    ...       ...
578 2020-01-01                     Quebec          Deforestation  331423.0
572 2020-01-01              New Brunswick          Deforestation  164728.0
577 2020-01-01       Prince Edward Island          Deforestation    5641.0
575 2020-01-01                Nova Scotia          Deforestation   49285.0
573 2020-01-01  Newfoundland and Labrador          Deforestation   10072.0

[460 rows x 4 columns]


,Year,Silvicultural Activity,value
0,1975-01-01,Deforestation,0.03
1,1976-01-01,Deforestation,0.13
2,1977-01-01,Deforestation,0.19
3,1978-01-01,Deforestation,0.33
4,1979-01-01,Deforestation,0.41


In [141]:
cumlative = alt.Chart(
simp_data2
).mark_line(
    size=6,
    interpolate="monotone",
    strokeDash=(10,6)
).encode(
    alt.X("Year:T"),
    alt.Y("value:Q").title("Cumulative Deforestation (1,000,000 Acres)"),
    alt.Color('Silvicultural Activity')
)

In [167]:
chart_title = alt.TitleParams(
    'Harvesting and Reforestation in Canada',
    subtitle=[""],
    color="#122740",
    subtitleColor= "#326b77",
    fontSize=20,
    subtitleFontSize=15
)

alt.Chart(data1,title = chart_title)
selector = alt.selection_point(fields=["Year"])

Canada = alt.Chart(
    data1,title = chart_title
).mark_bar(
    size=15, cornerRadiusEnd = 4
).encode(
    alt.X("Year:T",axis=alt.Axis(labelAngle=-45,grid=False)).title('Year (1975 - 2020)')
    ,alt.Y("sum(value):Q").title('Area (100,000 Acres)')
    ,yOffset="year:T"
    ,color = alt.condition(
        selector, alt.Color(
        'Silvicultural Activity:N',scale=alt.Scale(
            scheme=('#f9dd04','#0420f9')
        )
        ).legend(
            orient="top-left",
            labelFontSize=14,
            labelColor="#326b77",
            titleFontSize=16,
            titleColor="#122740",
            offset=30
        ),alt.value(
            'lightblue'
        )
    )
    ,tooltip = ("Year","sum(value):Q")
).properties(
    width=750, height=500
).add_params(
    selector
)

text = alt.Chart().mark_text(
    align="left",
    baseline="bottom",
    fontSize=14,
    fontWeight=600,
    color='#007bff',
    href="http://nfdp.ccfm.org/en/download.php"
).encode(
    x=alt.value(485),  # pixels from left
    y=alt.value(545),  # pixels from top
    text=alt.value("http://nfdp.ccfm.org/en/download.php")
    
)


provinces = alt.Chart(
    data1
).mark_bar(size=50, cornerRadiusEnd = 14).encode(
    alt.X('Provinces',sort='-y', axis=alt.Axis(labelAngle=-45)).title('Provinces')
    ,alt.Y('value:Q',sort='x').title('Area (100,000 Acres)')
    ,tooltip = ('Year','Provinces','value')
    ,color =  'Silvicultural Activity:N'
).properties(
    width=750, height=350
).transform_filter(
    selector
)

second_chart = alt.vconcat(alt.layer(Canada, cumlative).resolve_scale(
    y='independent'
),alt.layer(text,provinces)).configure_axis(
    labelFontSize=14,
    titleFontSize=20,
    titleColor="#122740",
    labelColor="#326b77"
)

second_chart.save('Harvesting and Reforestation in Canada (Final Draft).html')

second_chart

alt.VConcatChart(...)